# Overview

Testing 

* HDF5 output with Retiga
* Control of the shutter

# Hardware definition

* Retiga detector
* Aerotech stage
* 6IDD shutter

In [15]:
# workaround, not necessary in user production
import sys
sys.path.insert(0,'../../beamline/dev/jupyter-ht-hedm/')

In [16]:
from seisidd.devices.detectors import RetigaDetectorCam6IDD
from seisidd.devices.detectors import HDF5Plugin6IDD

from ophyd   import AreaDetector
from ophyd   import ADComponent
from ophyd   import SingleTrigger
from ophyd   import ProcessPlugin
from ophyd   import TIFFPlugin
from ophyd   import TransformPlugin

## Define Retiga

In [17]:
class RetigaDetector(SingleTrigger, AreaDetector):
    """PointGrey Detector used at 6-ID-D@APS for tomo and nf-HEDM"""

    cam1   = ADComponent(RetigaDetectorCam6IDD, suffix="cam1:" )  # camera
    proc1 = ADComponent(ProcessPlugin,         suffix="Proc1:")  # processing
    tiff1 = ADComponent(TIFFPlugin,            suffix="TIFF1:")  # tiff output
    hdf1  = ADComponent(HDF5Plugin6IDD,        suffix="HDF1:" )  # HDF5 output
    trans1 = ADComponent(TransformPlugin, suffix='Trans1:')

det = RetigaDetector('QIMAGE2:', name='det')

det.describe()

OrderedDict()

## Define Aerotech stage

In [78]:
from epics import caget, caput

caput("6idhedms1:m1.SET", 0)

1

In [129]:
from ophyd   import    Device
from ophyd   import    MotorBundle
from ophyd   import    Component as Cpt
from ophyd   import    EpicsMotor
from ophyd   import    EpicsSignal
from ophyd   import    EpicsSignalRO

class AeroRot(EpicsMotor):
    dial_readback  = Cpt(EpicsSignalRO,".DRBV", name = 'dial_readback')
    dial_value     = Cpt(EpicsSignal, ".DVAL", name = 'dial_value')
    _motor_cal_set = Cpt(EpicsSignal, ".SET",  name = '_motor_cal_set')
    _off_value     = Cpt(EpicsSignal, ".OFF",  name = '_off_value')
    
    

class StageAero(MotorBundle):
    """
    Motor stacks used for HT-HEDM

        ___________________________________
        |   fine translation:  kx,ky,kz   |
        |   fine tilt: kx_tilt, kz_tilt   |
        ===================================
        |    air-bearing rotation: rot    |
        ===================================
        |  coarse translation below Aero: | 
        |     x_base, y_base, z_base      |
        -----------------------------------

    """

    #   TODO:
    #   update with acutal PV
    ky      = Component(EpicsMotor, "6idhedm:m28", name='ky')  # y motion with kohzu stage
    kz      = Component(EpicsMotor, "6idhedm:m29", name='kz')  # z motion with kohzu stage
    kx      = Component(EpicsMotor, "6idhedm:m30", name='kx')  # x motion with kohzu stage

    rot     = Component(AeroRot, "6idhedms1:m1", name='rot')    # rotation with aero stage


In [135]:
tomostage = StageAero(name = 'tomostage')

tomostage.rot._motor_cal_set.put(1)
#_current_rot_pos = tomostage.rot._dialreadback.get()
tomostage.rot.dial_value.put(tomostage.rot.dial_readback.get()-360)
tomostage.rot._off_value.put(0)
tomostage.rot._motor_cal_set.put(0)

In [127]:

tomostage.rot._motor_cal_set.put(0)

## Shutters

In [39]:
import bluesky
import numpy as np
from   bluesky.suspenders  import SuspendFloor
from   apstools.devices    import ApsPssShutterWithStatus
from ophyd   import    EpicsMotor


D_shutter = ApsPssShutterWithStatus(
                "PA:06ID:",                          
                "PA:06ID:STA_D_SDS_OPEN_PL",        
                name="D_shutter",
                )

suspend_shutter = SuspendFloor(D_shutter.pss_state, 1)
shutter_suspender = suspend_shutter


fast_shutter = EpicsMotor("6idhedm:m7", name='fast_shutter')



# Set up Tomo scan plan

Due to the issue we have with the shutter controls, we cannot preform the automated scan from here.

In [113]:
# -- step scan
def reset():
    yield from bps.mv(tomostage.rot._motor_cal_set, 1)
    yield from bps.mv(tomostage.rot.dial_value,     tomostage.rot.dial_readback.get()-360)
    yield from bps.mv(tomostage.rot._off_value,     0)
    yield from bps.mv(tomostage.rot._motor_cal_set, 0)

    
def step_scan(cfg_tomo):
    """
    Collect projections with step motion
    """

    # TODO:
    # the fields need to be updated for 6-ID-D
    yield from bps.mv(det.hdf1.nd_array_port, 'PROC1')
    yield from bps.mv(det.tiff1.nd_array_port, 'PROC1') 
    yield from bps.mv(det.proc1.enable, 1)
    yield from bps.mv(det.proc1.enable_filter, 1)
    yield from bps.mv(det.proc1.filter_type, 'Average')
    yield from bps.mv(det.proc1.reset_filter, 1)
    yield from bps.mv(det.proc1.num_filter, cfg_tomo['n_frames'])
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames'])

    angs = np.arange(
        cfg_tomo['omega_start'], 
        cfg_tomo['omega_end']+cfg_tomo['omega_step']/2,
        cfg_tomo['omega_step'],
    )
    for ang in angs:
        yield from bps.checkpoint()
        if ang<-180 or ang>180:
            yield from reset()
        yield from bps.mv(tomostage.rot, ang)
        yield from bps.mv(fast_shutter, 13)
        yield from bps.trigger_and_read([det])
        yield from bps.mv(fast_shutter, 14)


# -- white field
def collect_white_field(cfg_tomo, atfront=True):
    """
    Collect white/flat field images by moving the sample out of the FOV
    """  
    # move sample out of the way
    # TODO:
    # the details and fields need to be updated for 6-ID-D
    _x = cfg_tomo['fronte_white_kx'] if atfront else cfg_tomo['back_white_kx']
    _z = cfg_tomo['fronte_white_kz'] if atfront else cfg_tomo['back_white_kz']
    yield from bps.mv(tomostage.kx, _x)  #update with correct motor name
    yield from bps.mv(tomostage.kz, _z)

    # setup detector
    # TODO:
    # actual implementation need to be for 6-ID-D
    yield from bps.mv(det.hdf1.nd_array_port, 'PROC1')
    yield from bps.mv(det.tiff1.nd_array_port, 'PROC1') 
    yield from bps.mv(det.proc1.enable, 1)
    yield from bps.mv(det.proc1.enable_filter, 1)
    yield from bps.mv(det.proc1.filter_type, 'Average')
    yield from bps.mv(det.proc1.reset_filter, 1)
    yield from bps.mv(det.proc1.num_filter, cfg_tomo['n_frames'])
    yield from bps.mv(det.cam1.trigger_mode, "Software")
    yield from bps.mv(det.cam1.image_mode, "Multiple")
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames']*cfg_tomo['n_white'])
    yield from bps.mv(fast_shutter, 13)
    yield from bps.trigger_and_read([det])
    yield from bps.mv(fast_shutter, 14)
    # move sample back to FOV
    # NOTE:
    # not sure is this will work or not...
    # TODO:
    #   need to update all the motor names according to StageAero
    yield from bps.mv(tomostage.kx, cfg_tomo['initial_kx'])
    yield from bps.mv(tomostage.kz, cfg_tomo['initial_kz'])
    

# -- dark field
def collect_dark_field(cfg_tomo):
    """
    Collect dark field images by close the shutter
    """
    # TODO:
    #   Need to toggle Fast shutter

    yield from bps.mv(det.hdf1.nd_array_port, 'PROC1')
    yield from bps.mv(det.tiff1.nd_array_port, 'PROC1') 
    yield from bps.mv(det.proc1.enable, 1)
    yield from bps.mv(det.proc1.enable_filter, 1)
    yield from bps.mv(det.proc1.filter_type, 'Average')
    yield from bps.mv(det.proc1.reset_filter, 1)
    yield from bps.mv(det.proc1.num_filter, cfg_tomo['n_frames'])
    yield from bps.mv(det.cam1.trigger_mode, "Software")
    yield from bps.mv(det.cam1.image_mode, "Multiple")
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames']*cfg_tomo['n_dark'])
    yield from bps.mv(fast_shutter, 14)
    yield from bps.trigger_and_read([det])


# -- tomo scan
def tomo_scan(cfg):
    """
    Tomography scan plan based on given configuration
    """
    tomostage.kx.move(-1.460)
    tomostage.kz.move(0.030)
    tomostage.ky.move(16.353)
    
    
    fp = cfg['output']['fp']
    fn = cfg['output']['fn']

    #########################
    ## step 0: preparation ##
    #########################
    acquire_time   = cfg['tomo']['acquire_time']
    acquire_period = cfg['tomo']['acquire_period']
    n_white        = cfg['tomo']['n_white']
    n_dark         = cfg['tomo']['n_dark']
    angs = np.arange(
        cfg['tomo']['omega_start'], 
        cfg['tomo']['omega_end']+cfg['tomo']['omega_step']/2,
        cfg['tomo']['omega_step'],
    )
    n_projections = len(angs)
    cfg['tomo']['n_projections'] = n_projections
    cfg['tomo']['total_images']  = n_white + n_projections + n_white + n_dark


    # calculate slew speed for fly scan
    # https://github.com/decarlof/tomo2bm/blob/master/flir/libs/aps2bm_lib.py
    # TODO: considering blue pixels, use 2BM code as ref

    # need to make sure that the sample out position is the same for both front and back
    x0, z0 = tomostage.kx.position, tomostage.kz.position
    dfx, dfz = cfg['tomo']['sample_out_position']['kx'], cfg['tomo']['sample_out_position']['kz']
    rotang = np.radians(cfg['tomo']['omega_end']-cfg['tomo']['omega_start'])
    rotm = np.array([[ np.cos(rotang), np.sin(rotang)],
                        [-np.sin(rotang), np.cos(rotang)]])
    dbxz = np.dot(rotm, np.array([dfx, dfz]))
    dbx = dbxz[0] if abs(dbxz[0]) > 1e-8 else 0.0
    dbz = dbxz[1] if abs(dbxz[1]) > 1e-8 else 0.0
    # now put the value to dict
    cfg['tomo']['initial_kx']       = x0
    cfg['tomo']['initial_kz']       = z0
    cfg['tomo']['fronte_white_kx']  = x0 + dfx
    cfg['tomo']['fronte_white_kz']  = z0 + dfz
    cfg['tomo']['back_white_kx']    = x0 + dbx
    cfg['tomo']['back_white_kz']    = z0 + dbz

    ###############################################
    ## step 0.9: print out the cfg for user info ##
    ###############################################   
    caput("QIMAGE2:TIFF1:AutoSave", 1)
    @bpp.stage_decorator([det])
    @bpp.run_decorator()
    def scan_closure():
        # TODO:
        #   Somewhere we need to check the light status
        # open shutter for beam

        yield from bps.install_suspender(shutter_suspender)
        # config output
        for me in [det.tiff1, det.hdf1]:
            yield from bps.mv(me.file_path, fp)
            yield from bps.mv(me.file_name, fn)
            yield from bps.mv(me.file_write_mode, 2)
            yield from bps.mv(me.num_capture, cfg['tomo']['total_images'])
            yield from bps.mv(me.file_template, ".".join([r"%s%s_%06d",cfg['output']['type'].lower()]))    

        if cfg['output']['type'] in ['tif', 'tiff']:
            yield from bps.mv(det.tiff1.enable, 1)
            yield from bps.mv(det.tiff1.capture, 1)
            yield from bps.mv(det.hdf1.enable, 0)
        elif cfg['output']['type'] in ['hdf', 'hdf1', 'hdf5']:
            yield from bps.mv(det.tiff1.enable, 0)
            yield from bps.mv(det.hdf1.enable, 1)
            yield from bps.mv(det.hdf1.capture, 1)
        else:
            raise ValueError(f"Unsupported output type {cfg['output']['type']}")
        
#         # open main shutter
#         yield from bps.mv(D_shutter, "open")

        # collect front white field
        yield from bps.mv(det.cam1.frame_type, 0)  # for HDF5 dxchange data structure
        yield from collect_white_field(cfg['tomo'], atfront=True)

        # collect projections
        yield from bps.mv(det.cam1.frame_type, 1)  # for HDF5 dxchange data structure
        if cfg['tomo']['type'].lower() == 'step':
            # setting acquire_time and acquire_period
            yield from bps.mv(det.cam1.acquire_time, acquire_time)
            yield from bps.mv(det.cam1.acquire_period, acquire_period)
            # run step_scan
            yield from step_scan(cfg['tomo'])
        else:
            raise ValueError(f"Unsupported scan type: {cfg['tomo']['type']}")

        # collect back white field
        yield from bps.mv(det.cam1.frame_type, 2)  # for HDF5 dxchange data structure
        yield from collect_white_field(cfg['tomo'], atfront=False)

        # collect back dark field
        yield from bps.mv(det.cam1.frame_type, 3)  # for HDF5 dxchange data structure

#         # TODO: no shutter available for Sim testing
#         yield from bps.remove_suspender(shutter_suspender)
#         yield from bps.mv(D_shutter, "close")
#         yield from bps.mv(fast_shutter, 1)

        yield from collect_dark_field(cfg['tomo'])
        
        yield from bps.mv(det_mtr_x, 50)
        yield from bps.mv(tomostage.ky, 6.0)

    return (yield from scan_closure())

## Setup RunEngine

In [43]:
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky.simulators import summarize_plan
import bluesky.preprocessors as bpp
import bluesky.plan_stubs as bps

RE = bluesky.RunEngine({})
RE.subscribe(BestEffortCallback())

0

In [44]:
cfg_tomo = {
    'type':    'step',
    'n_white': 5,
    'n_dark':  5,
    'sample_out_position': {
        'kx':  -3,
        'kz':   0,
    },
    'acquire_time':   0.5,
    'acquire_period': 1.0, 
    'omega_step':     0.1,
    'omega_start':    -180,
    'omega_end':       35820,
    'n_frames':       1,   
}

cfg_output = {
    'fn':          'HEA1_3',
    'fp':          'Q:\\\\s6idd_dec19\\tomo_HEA1_3\\' ,  # match the ff scan HEA_3
    'type':        'tif',  
}

cfg = {
    'tomo': cfg_tomo,
    'output': cfg_output,
}

In [45]:
summarize_plan(tomo_scan(cfg))

=================================== Open Run ===================================
det_tiff1_file_path -> Q:\\s6idd_dec19\tomo_HEA1_3\
det_tiff1_file_name -> HEA1_3
det_tiff1_file_write_mode -> 2
det_tiff1_num_capture -> 360016
det_tiff1_file_template -> %s%s_%06d.tif
det_hdf1_file_path -> Q:\\s6idd_dec19\tomo_HEA1_3\
det_hdf1_file_name -> HEA1_3
det_hdf1_file_write_mode -> 2
det_hdf1_num_capture -> 360016
det_hdf1_file_template -> %s%s_%06d.tif
det_tiff1_enable -> 1
det_tiff1_capture -> 1
det_hdf1_enable -> 0
det_cam1_frame_type -> 0
tomostage_kx -> -4.460000000000001
tomostage_kz -> 0.03000000000000111
det_hdf1_nd_array_port -> PROC1
det_tiff1_nd_array_port -> PROC1
det_proc1_enable -> 1
det_proc1_enable_filter -> 1
det_proc1_filter_type -> Average
det_proc1_reset_filter -> 1
det_proc1_num_filter -> 1
det_cam1_trigger_mode -> Software
det_cam1_image_mode -> Multiple
det_cam1_num_images -> 5
fast_shutter -> 13
  Read ['det']
fast_shutter -> 14
tomostage_kx -> -1.4600000000000009
tomosta

KeyboardInterrupt: 

In [ ]:
RE(tomo_scan(cfg))

Transient Scan ID: 1     Time: 2019-12-12 21:06:58
Persistent Unique Scan ID: '6c13d243-e34b-4f6f-ba7b-babb4f2f30f6'


/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/epics/pv.py:110: UserWarning: The `context` kwarg for epics.get_pv() is deprecated. New PVs will _not_ be created in the requested context.
  'The `context` kwarg for epics.get_pv() is deprecated. New PVs '


New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 21:07:08.8 |
|         2 | 21:07:52.4 |
|         3 | 21:07:54.8 |
|         4 | 21:07:57.2 |
|         5 | 21:07:59.6 |
|         6 | 21:08:02.0 |
|         7 | 21:08:04.4 |
|         8 | 21:08:06.8 |
|         9 | 21:08:09.2 |
|        10 | 21:08:11.6 |
|        11 | 21:08:14.0 |
|        12 | 21:08:16.4 |
|        13 | 21:08:18.8 |


/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/epics/ca.py:1507: UserWarning: ca.get('6idhedm:m7.HLS') timed out after 1.00 seconds.
  warnings.warn(msg % (name(chid), timeout))


|        14 | 21:08:22.2 |
|        15 | 21:08:24.6 |
|        16 | 21:08:27.2 |
|        17 | 21:08:29.6 |
|        18 | 21:08:32.0 |
|        19 | 21:08:34.4 |
|        20 | 21:08:36.8 |
|        21 | 21:08:39.2 |
|        22 | 21:08:41.6 |
|        23 | 21:08:44.0 |
|        24 | 21:08:46.4 |
|        25 | 21:08:48.8 |
|        26 | 21:08:51.2 |
|        27 | 21:08:53.6 |
|        28 | 21:08:56.0 |
|        29 | 21:08:58.4 |
|        30 | 21:09:00.8 |
|        31 | 21:09:03.2 |
|        32 | 21:09:05.6 |
|        33 | 21:09:08.0 |
|        34 | 21:09:10.4 |
|        35 | 21:09:12.8 |
|        36 | 21:09:15.2 |
|        37 | 21:09:17.6 |
|        38 | 21:09:20.0 |
|        39 | 21:09:22.4 |
|        40 | 21:09:24.8 |
|        41 | 21:09:27.2 |
|        42 | 21:09:29.6 |
|        43 | 21:09:32.0 |
|        44 | 21:09:34.4 |
|        45 | 21:09:36.8 |
|        46 | 21:09:39.2 |
|        47 | 21:09:41.6 |
|        48 | 21:09:44.0 |
|        49 | 21:09:46.4 |
+-----------+------------+
|

In [ ]:
from epics import caput    
    
caput("QIMAGE2:TIFF1:AutoSave", 0)

In [47]:
RE.abort()

Aborting: running cleanup and marking exit_status as 'abort'...
+-----------+------------+
generator tomo_scan ['6c13d243'] (scan num: 1)



[E 08:51:45.334 run_engine:1469] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1345, in _run
        msg = self._plan_stack[-1].send(resp)
      File "<ipython-input-42-35bc7e3791a6>", line 208, in tomo_scan
        return (yield from scan_closure())
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/utils.py", line 1043, in dec_inner
        return (yield from plan)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/preprocessors.py", line 940, in stage_wrapper
        return (yield from finalize_wrapper(inner(), unstage_devices()))
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/preprocessors.p

('6c13d243-e34b-4f6f-ba7b-babb4f2f30f6',)

The aerotech somehow ending up in the negative range, taking out the imaging part and just continously rotating the motor.

In [48]:
def dmmmy_step_scan(cfg_tomo):
    """rotate aero tech without triggering detector"""
    angs = np.arange(
        cfg_tomo['omega_start'], 
        cfg_tomo['omega_end']+cfg_tomo['omega_step']/2,
        cfg_tomo['omega_step'],
    )
    for ang in angs:
        yield from bps.checkpoint()
        yield from bps.mv(tomostage.rot, ang)
        yield from bps.mv(fast_shutter, 13)
        yield from bps.sleep(0.5)  # mimicing image acquisition
        yield from bps.mv(fast_shutter, 14)

def test_aerotech_slipring(cfg):
    """
    Monitoring the position while contiously rotating aerotech
    """
    
    tomostage.kx.move(-1.460)
    tomostage.kz.move(0.030)
    tomostage.ky.move(16.353)
    
    
    fp = cfg['output']['fp']
    fn = cfg['output']['fn']

    #########################
    ## step 0: preparation ##
    #########################
    acquire_time   = cfg['tomo']['acquire_time']
    acquire_period = cfg['tomo']['acquire_period']
    n_white        = cfg['tomo']['n_white']
    n_dark         = cfg['tomo']['n_dark']
    angs = np.arange(
        cfg['tomo']['omega_start'], 
        cfg['tomo']['omega_end']+cfg['tomo']['omega_step']/2,
        cfg['tomo']['omega_step'],
    )
    n_projections = len(angs)
    cfg['tomo']['n_projections'] = n_projections
    cfg['tomo']['total_images']  = n_white + n_projections + n_white + n_dark
    
    # calculate slew speed for fly scan
    # https://github.com/decarlof/tomo2bm/blob/master/flir/libs/aps2bm_lib.py
    # TODO: considering blue pixels, use 2BM code as ref

    # need to make sure that the sample out position is the same for both front and back
    x0, z0 = tomostage.kx.position, tomostage.kz.position
    dfx, dfz = cfg['tomo']['sample_out_position']['kx'], cfg['tomo']['sample_out_position']['kz']
    rotang = np.radians(cfg['tomo']['omega_end']-cfg['tomo']['omega_start'])
    rotm = np.array([[ np.cos(rotang), np.sin(rotang)],
                        [-np.sin(rotang), np.cos(rotang)]])
    dbxz = np.dot(rotm, np.array([dfx, dfz]))
    dbx = dbxz[0] if abs(dbxz[0]) > 1e-8 else 0.0
    dbz = dbxz[1] if abs(dbxz[1]) > 1e-8 else 0.0
    # now put the value to dict
    cfg['tomo']['initial_kx']       = x0
    cfg['tomo']['initial_kz']       = z0
    cfg['tomo']['fronte_white_kx']  = x0 + dfx
    cfg['tomo']['fronte_white_kz']  = z0 + dfz
    cfg['tomo']['back_white_kx']    = x0 + dbx
    cfg['tomo']['back_white_kz']    = z0 + dbz
    
    @bpp.run_decorator()
    def scan_closure():
        # config output
        if cfg['tomo']['type'].lower() == 'step':
            yield from dmmmy_step_scan(cfg['tomo'])
        else:
            raise ValueError(f"Unsupported scan type: {cfg['tomo']['type']}")

        # restore position
        yield from bps.mv(det_mtr_x, 50)
        yield from bps.mv(tomostage.ky, 6.0)

    return (yield from scan_closure())

In [49]:
from bluesky.callbacks import LiveTable

In [56]:
cfg_tomo_aerotech_test = {
    'type':    'step',
    'n_white': 5,
    'n_dark':  5,
    'sample_out_position': {
        'kx':  -3,
        'kz':   0,
    },
    'acquire_time':   0.5,
    'acquire_period': 1.0, 
    'omega_step':     0.1,
    'omega_start':    -180,
    'omega_end':       35820,
    'n_frames':       1,   
}

cfg_output = {
    'fn':          'HEA1_3',
    'fp':          'Q:\\\\s6idd_dec19\\tomo_HEA1_3\\' ,  # match the ff scan HEA_3
    'type':        'tif',  
}

cfg = {
    'tomo': cfg_tomo,
    'output': cfg_output,
}

In [51]:
RE(test_aerotech_slipring(cfg_tomo_aerotech_test), LiveTable([tomostage.rot]))

At least one suspender has tripped. The plan will begin when all suspenders are ready. Justification:
    1. Signal D_shutter_pss_state = 0 is below 1

Suspending... To get to the prompt, hit Ctrl-C twice to pause.
A 'deferred pause' has been requested. The RunEngine will pause at the next checkpoint. To pause immediately, hit Ctrl+C again in the next 10 seconds.
Deferred pause acknowledged. Continuing to checkpoint.
trying a second time
Pausing...


RunEngineInterrupted: 
Your RunEngine is entering a paused state. These are your options for changing
the state of the RunEngine:

RE.resume()    Resume the plan.
RE.abort()     Perform cleanup, then kill plan. Mark exit_stats='aborted'.
RE.stop()      Perform cleanup, then kill plan. Mark exit_status='success'.
RE.halt()      Emergency Stop: Do not perform cleanup --- just stop.


In [52]:
RE.remove_suspender(shutter_suspender)

In [53]:
RE(test_aerotech_slipring(cfg_tomo_aerotech_test), LiveTable([tomostage.rot]))

RuntimeError: The RunEngine is in a paused state

In [54]:
RE.abort()

Aborting: running cleanup and marking exit_status as 'abort'...
[E 09:08:10.794 run_engine:1469] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1322, in _run
        stashed_exception or resp)
      File "<ipython-input-48-bccca591d294>", line 15, in test_aerotech_slipring
        def test_aerotech_slipring(cfg):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1322, in _run
        stashed_exception or resp)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/utils.py", line 139, in single_gen
        return (yield msg)
    bluesky.utils.RequestAbort


()

In [55]:
RE(test_aerotech_slipring(cfg_tomo_aerotech_test), LiveTable([tomostage.rot]))

[E 09:08:18.122 run_engine:1477] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1345, in _run
        msg = self._plan_stack[-1].send(resp)
      File "<ipython-input-48-bccca591d294>", line 25, in test_aerotech_slipring
        fp = cfg['output']['fp']
    KeyError: 'output'


KeyError: 'output'

In [57]:
cfg_tomo = {
    'type':    'step',
    'n_white': 5,
    'n_dark':  5,
    'sample_out_position': {
        'kx':  -3,
        'kz':   0,
    },
    'acquire_time':   0.5,
    'acquire_period': 1.0, 
    'omega_step':     0.1,
    'omega_start':    -180,
    'omega_end':       35820,
    'n_frames':       1,   
}

cfg_output = {
    'fn':          'HEA1_3',
    'fp':          'Q:\\\\s6idd_dec19\\tomo_HEA1_3\\' ,  # match the ff scan HEA_3
    'type':        'tif',  
}

cfg_aerotech_test = {
    'tomo': cfg_tomo,
    'output': cfg_output,
}

In [58]:
RE(test_aerotech_slipring(cfg_aerotech_test), LiveTable([tomostage.rot]))

Transient Scan ID: 2     Time: 2019-12-13 09:09:53
Persistent Unique Scan ID: '7ad88964-01f3-4257-b366-8a830eb71a4a'
A 'deferred pause' has been requested. The RunEngine will pause at the next checkpoint. To pause immediately, hit Ctrl+C again in the next 10 seconds.Deferred pause acknowledged. Continuing to checkpoint.

trying a second time
Pausing...


RunEngineInterrupted: 
Your RunEngine is entering a paused state. These are your options for changing
the state of the RunEngine:

RE.resume()    Resume the plan.
RE.abort()     Perform cleanup, then kill plan. Mark exit_stats='aborted'.
RE.stop()      Perform cleanup, then kill plan. Mark exit_status='success'.
RE.halt()      Emergency Stop: Do not perform cleanup --- just stop.


In [59]:
RE.abort()

Aborting: running cleanup and marking exit_status as 'abort'...



generator test_aerotech_slipring ['7ad88964'] (scan num: 2)
[E 09:21:48.306 run_engine:1469] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1345, in _run
        msg = self._plan_stack[-1].send(resp)
      File "<ipython-input-48-bccca591d294>", line 77, in test_aerotech_slipring
        return (yield from scan_closure())
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/utils.py", line 1043, in dec_inner
        return (yield from plan)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/preprocessors.py", line 327, in run_wrapper
        else_plan=close_run)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/preprocessors.py", line 563, in contingency_wrapper
        ret

('7ad88964-01f3-4257-b366-8a830eb71a4a',)

In [60]:
tomostage.kx.describe()


OrderedDict([('tomostage_kx',
              {'source': 'PV:6idhedm:m30.RBV',
               'dtype': 'number',
               'shape': [],
               'precision': 5,
               'units': 'degrees',
               'lower_ctrl_limit': -9.459,
               'upper_ctrl_limit': 9.940999999999999}),
             ('tomostage_kx_user_setpoint',
              {'source': 'PV:6idhedm:m30.VAL',
               'dtype': 'number',
               'shape': [],
               'precision': 5,
               'units': 'degrees',
               'lower_ctrl_limit': -9.459,
               'upper_ctrl_limit': 9.940999999999999})])

In [61]:
mtrkx = tomostage.kx

In [69]:
from epics import caget, caput

caget("6idhedm:m30.DVAL")


-1.7009999999999992

In [72]:
tomostage.rot.

['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu']

In [139]:
def reset():
    yield from bps.mv(tomostage.rot._motor_cal_set, 1)
    yield from bps.mv(tomostage.rot.dial_value,     tomostage.rot.dial_readback.get()-360)
    yield from bps.mv(tomostage.rot._off_value,     0)
    yield from bps.mv(tomostage.rot._motor_cal_set, 0)

    
def dmmmy_step_scan(cfg_tomo):
    """rotate aero tech without triggering detector"""
    angs = np.arange(
        cfg_tomo['omega_start'], 
        cfg_tomo['omega_end']+cfg_tomo['omega_step']/2,
        cfg_tomo['omega_step'],
    )
    for ang in angs:
        yield from bps.checkpoint()
        yield from bps.mv(tomostage.rot, ang)
        yield from bps.mv(fast_shutter, 13)
        yield from bps.sleep(0.5)  # mimicing image acquisition
        yield from bps.mv(fast_shutter, 14)

def test_aerotech_slipring(cfg):
    """
    Monitoring the position while contiously rotating aerotech
    """
    
    tomostage.kx.move(-1.460)
    tomostage.kz.move(0.030)
    tomostage.ky.move(16.353)
    
    
    fp = cfg['output']['fp']
    fn = cfg['output']['fn']

    #########################
    ## step 0: preparation ##
    #########################
    acquire_time   = cfg['tomo']['acquire_time']
    acquire_period = cfg['tomo']['acquire_period']
    n_white        = cfg['tomo']['n_white']
    n_dark         = cfg['tomo']['n_dark']
    angs = np.arange(
        cfg['tomo']['omega_start'], 
        cfg['tomo']['omega_end']+cfg['tomo']['omega_step']/2,
        cfg['tomo']['omega_step'],
    )
    n_projections = len(angs)
    cfg['tomo']['n_projections'] = n_projections
    cfg['tomo']['total_images']  = n_white + n_projections + n_white + n_dark
    
    # calculate slew speed for fly scan
    # https://github.com/decarlof/tomo2bm/blob/master/flir/libs/aps2bm_lib.py
    # TODO: considering blue pixels, use 2BM code as ref

    # need to make sure that the sample out position is the same for both front and back
    x0, z0 = tomostage.kx.position, tomostage.kz.position
    dfx, dfz = cfg['tomo']['sample_out_position']['kx'], cfg['tomo']['sample_out_position']['kz']
    rotang = np.radians(cfg['tomo']['omega_end']-cfg['tomo']['omega_start'])
    rotm = np.array([[ np.cos(rotang), np.sin(rotang)],
                        [-np.sin(rotang), np.cos(rotang)]])
    dbxz = np.dot(rotm, np.array([dfx, dfz]))
    dbx = dbxz[0] if abs(dbxz[0]) > 1e-8 else 0.0
    dbz = dbxz[1] if abs(dbxz[1]) > 1e-8 else 0.0
    # now put the value to dict
    cfg['tomo']['initial_kx']       = x0
    cfg['tomo']['initial_kz']       = z0
    cfg['tomo']['fronte_white_kx']  = x0 + dfx
    cfg['tomo']['fronte_white_kz']  = z0 + dfz
    cfg['tomo']['back_white_kx']    = x0 + dbx
    cfg['tomo']['back_white_kz']    = z0 + dbz
    
    @bpp.run_decorator()
    def scan_closure():
        # config output
        if cfg['tomo']['type'].lower() == 'step':
            yield from dmmmy_step_scan(cfg['tomo'])
        else:
            raise ValueError(f"Unsupported scan type: {cfg['tomo']['type']}")

        # restore position
        yield from bps.mv(det_mtr_x, 50)
        yield from bps.mv(tomostage.ky, 6.0)

    return (yield from scan_closure())

def repeat(n_turns, cfg):
    for _ in range(n_turns):
        yield from test_aerotech_slipring(cfg)
        yield from reset()


In [141]:
cfg_tomo = {
    'type':    'step',
    'n_white': 5,
    'n_dark':  5,
    'sample_out_position': {
        'kx':  -3,
        'kz':   0,
    },
    'acquire_time':   0.5,
    'acquire_period': 1.0, 
    'omega_step':     0.1,
    'omega_start':    -180,
    'omega_end':       35820,
    'n_frames':       1,   
}

cfg_output = {
    'fn':          'HEA1_3',
    'fp':          'Q:\\\\s6idd_dec19\\tomo_HEA1_3\\' ,  # match the ff scan HEA_3
    'type':        'tif',  
}

cfg_aerotech_test = {
    'tomo': cfg_tomo,
    'output': cfg_output,
}

In [140]:
RE(repeat(2, cfg_aerotech_test), LiveTable([tomostage.rot]))

Transient Scan ID: 3     Time: 2019-12-13 12:17:27
Persistent Unique Scan ID: '3965d15f-859b-43c8-8c81-15dc88f4e0b2'
A 'deferred pause' has been requested. The RunEngine will pause at the next checkpoint. To pause immediately, hit Ctrl+C again in the next 10 seconds.
Deferred pause acknowledged. Continuing to checkpoint.
trying a second time
Pausing...


RunEngineInterrupted: 
Your RunEngine is entering a paused state. These are your options for changing
the state of the RunEngine:

RE.resume()    Resume the plan.
RE.abort()     Perform cleanup, then kill plan. Mark exit_stats='aborted'.
RE.stop()      Perform cleanup, then kill plan. Mark exit_status='success'.
RE.halt()      Emergency Stop: Do not perform cleanup --- just stop.


In [142]:
RE.abort()

Aborting: running cleanup and marking exit_status as 'abort'...



generator repeat ['3965d15f'] (scan num: 3)
[E 12:18:35.700 run_engine:1469] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1345, in _run
        msg = self._plan_stack[-1].send(resp)
      File "<ipython-input-139-0f7fafc7c7de>", line 88, in repeat
        yield from test_aerotech_slipring(cfg)
      File "<ipython-input-139-0f7fafc7c7de>", line 84, in test_aerotech_slipring
        return (yield from scan_closure())
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/utils.py", line 1043, in dec_inner
        return (yield from plan)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/preprocessors.py", line 327, in run_wrapper
        else_plan=close_run)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/l

('3965d15f-859b-43c8-8c81-15dc88f4e0b2',)

In [143]:
cfg_tomo = {
    'type':    'step',
    'n_white': 5,
    'n_dark':  5,
    'sample_out_position': {
        'kx':  -3,
        'kz':   0,
    },
    'acquire_time':   0.5,
    'acquire_period': 1.0, 
    'omega_step':     10,
    'omega_start':    -180,
    'omega_end':       35820,
    'n_frames':       1,   
}

cfg_output = {
    'fn':          'HEA1_3',
    'fp':          'Q:\\\\s6idd_dec19\\tomo_HEA1_3\\' ,  # match the ff scan HEA_3
    'type':        'tif',  
}

cfg_aerotech_test = {
    'tomo': cfg_tomo,
    'output': cfg_output,
}

RE(repeat(2, cfg_aerotech_test), LiveTable([tomostage.rot]))

Transient Scan ID: 4     Time: 2019-12-13 12:19:08
Persistent Unique Scan ID: '1c5d10ed-2847-4ac6-a208-e6e37dd466a2'
A 'deferred pause' has been requested. The RunEngine will pause at the next checkpoint. To pause immediately, hit Ctrl+C again in the next 10 seconds.Deferred pause acknowledged. Continuing to checkpoint.

trying a second time
Pausing...


RunEngineInterrupted: 
Your RunEngine is entering a paused state. These are your options for changing
the state of the RunEngine:

RE.resume()    Resume the plan.
RE.abort()     Perform cleanup, then kill plan. Mark exit_stats='aborted'.
RE.stop()      Perform cleanup, then kill plan. Mark exit_status='success'.
RE.halt()      Emergency Stop: Do not perform cleanup --- just stop.


In [144]:
RE.abort()

Aborting: running cleanup and marking exit_status as 'abort'...



generator repeat ['1c5d10ed'] (scan num: 4)
[E 12:21:48.763 run_engine:1469] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1345, in _run
        msg = self._plan_stack[-1].send(resp)
      File "<ipython-input-139-0f7fafc7c7de>", line 88, in repeat
        yield from test_aerotech_slipring(cfg)
      File "<ipython-input-139-0f7fafc7c7de>", line 84, in test_aerotech_slipring
        return (yield from scan_closure())
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/utils.py", line 1043, in dec_inner
        return (yield from plan)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/preprocessors.py", line 327, in run_wrapper
        else_plan=close_run)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/l

('1c5d10ed-2847-4ac6-a208-e6e37dd466a2',)

In [145]:
cfg_tomo = {
    'type':    'step',
    'n_white': 5,
    'n_dark':  5,
    'sample_out_position': {
        'kx':  -3,
        'kz':   0,
    },
    'acquire_time':   0.5,
    'acquire_period': 1.0, 
    'omega_step':     10,
    'omega_start':    -180,
    'omega_end':       180,
    'n_frames':       1,   
}

cfg_output = {
    'fn':          'HEA1_3',
    'fp':          'Q:\\\\s6idd_dec19\\tomo_HEA1_3\\' ,  # match the ff scan HEA_3
    'type':        'tif',  
}

cfg_aerotech_test = {
    'tomo': cfg_tomo,
    'output': cfg_output,
}

RE(repeat(2, cfg_aerotech_test), LiveTable([tomostage.rot]))

Transient Scan ID: 5     Time: 2019-12-13 12:22:08
Persistent Unique Scan ID: '884353a5-03d9-4406-b589-7c77af529334'



generator repeat ['884353a5'] (scan num: 5)
[E 12:25:04.018 run_engine:1477] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1345, in _run
        msg = self._plan_stack[-1].send(resp)
      File "<ipython-input-139-0f7fafc7c7de>", line 88, in repeat
        yield from test_aerotech_slipring(cfg)
      File "<ipython-input-139-0f7fafc7c7de>", line 84, in test_aerotech_slipring
        return (yield from scan_closure())
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/utils.py", line 1043, in dec_inner
        return (yield from plan)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/preprocessors.py", line 327, in run_wrapper
        else_plan=close_run)
      F

NameError: name 'det_mtr_x' is not defined

In [146]:
def reset():
    yield from bps.mv(tomostage.rot._motor_cal_set, 1)
    yield from bps.mv(tomostage.rot.dial_value,     tomostage.rot.dial_readback.get()-360)
    yield from bps.mv(tomostage.rot._off_value,     0)
    yield from bps.mv(tomostage.rot._motor_cal_set, 0)

    
def dmmmy_step_scan(cfg_tomo):
    """rotate aero tech without triggering detector"""
    angs = np.arange(
        cfg_tomo['omega_start'], 
        cfg_tomo['omega_end']+cfg_tomo['omega_step']/2,
        cfg_tomo['omega_step'],
    )
    for ang in angs:
        yield from bps.checkpoint()
        yield from bps.mv(tomostage.rot, ang)
        yield from bps.mv(fast_shutter, 13)
        yield from bps.sleep(0.5)  # mimicing image acquisition
        yield from bps.mv(fast_shutter, 14)

def test_aerotech_slipring(cfg):
    """
    Monitoring the position while contiously rotating aerotech
    """
    
    tomostage.kx.move(-1.460)
    tomostage.kz.move(0.030)
    tomostage.ky.move(16.353)
    
    
    fp = cfg['output']['fp']
    fn = cfg['output']['fn']

    #########################
    ## step 0: preparation ##
    #########################
    acquire_time   = cfg['tomo']['acquire_time']
    acquire_period = cfg['tomo']['acquire_period']
    n_white        = cfg['tomo']['n_white']
    n_dark         = cfg['tomo']['n_dark']
    angs = np.arange(
        cfg['tomo']['omega_start'], 
        cfg['tomo']['omega_end']+cfg['tomo']['omega_step']/2,
        cfg['tomo']['omega_step'],
    )
    n_projections = len(angs)
    cfg['tomo']['n_projections'] = n_projections
    cfg['tomo']['total_images']  = n_white + n_projections + n_white + n_dark
    
    # calculate slew speed for fly scan
    # https://github.com/decarlof/tomo2bm/blob/master/flir/libs/aps2bm_lib.py
    # TODO: considering blue pixels, use 2BM code as ref

    # need to make sure that the sample out position is the same for both front and back
    x0, z0 = tomostage.kx.position, tomostage.kz.position
    dfx, dfz = cfg['tomo']['sample_out_position']['kx'], cfg['tomo']['sample_out_position']['kz']
    rotang = np.radians(cfg['tomo']['omega_end']-cfg['tomo']['omega_start'])
    rotm = np.array([[ np.cos(rotang), np.sin(rotang)],
                        [-np.sin(rotang), np.cos(rotang)]])
    dbxz = np.dot(rotm, np.array([dfx, dfz]))
    dbx = dbxz[0] if abs(dbxz[0]) > 1e-8 else 0.0
    dbz = dbxz[1] if abs(dbxz[1]) > 1e-8 else 0.0
    # now put the value to dict
    cfg['tomo']['initial_kx']       = x0
    cfg['tomo']['initial_kz']       = z0
    cfg['tomo']['fronte_white_kx']  = x0 + dfx
    cfg['tomo']['fronte_white_kz']  = z0 + dfz
    cfg['tomo']['back_white_kx']    = x0 + dbx
    cfg['tomo']['back_white_kz']    = z0 + dbz
    
    @bpp.run_decorator()
    def scan_closure():
        # config output
        if cfg['tomo']['type'].lower() == 'step':
            yield from dmmmy_step_scan(cfg['tomo'])
        else:
            raise ValueError(f"Unsupported scan type: {cfg['tomo']['type']}")

        # restore position
#         yield from bps.mv(det_mtr_x, 50)
        yield from bps.mv(tomostage.ky, 6.0)

    return (yield from scan_closure())

def repeat(n_turns, cfg):
    for _ in range(n_turns):
        yield from test_aerotech_slipring(cfg)
        yield from reset()


In [147]:
RE.abort()

TransitionError: RunEngine is already idle.

In [148]:
tomostage.rot._motor_cal_set.put(1)
tomostage.rot.dial_value.put(tomostage.rot.dial_readback.get()-360)
tomostage.rot._off_value.put(0)
tomostage.rot._motor_cal_set.put(0)

In [149]:
cfg_tomo = {
    'type':    'step',
    'n_white': 5,
    'n_dark':  5,
    'sample_out_position': {
        'kx':  -3,
        'kz':   0,
    },
    'acquire_time':   0.5,
    'acquire_period': 1.0, 
    'omega_step':     10,
    'omega_start':    -180,
    'omega_end':       180,
    'n_frames':       1,   
}

cfg_output = {
    'fn':          'HEA1_3',
    'fp':          'Q:\\\\s6idd_dec19\\tomo_HEA1_3\\' ,  # match the ff scan HEA_3
    'type':        'tif',  
}

cfg_aerotech_test = {
    'tomo': cfg_tomo,
    'output': cfg_output,
}

RE(repeat(2, cfg_aerotech_test), LiveTable([tomostage.rot]))

Transient Scan ID: 6     Time: 2019-12-13 12:27:19
Persistent Unique Scan ID: '7b55db60-d885-413e-8345-218940c92f03'



generator repeat ['7b55db60'] (scan num: 6)
Transient Scan ID: 7     Time: 2019-12-13 12:33:07
Persistent Unique Scan ID: '80530566-5bd8-4dd2-b2f0-ea75d8f210e4'



generator repeat ['80530566'] (scan num: 7)


('7b55db60-d885-413e-8345-218940c92f03',
 '80530566-5bd8-4dd2-b2f0-ea75d8f210e4')

In [150]:
RE(repeat(100, cfg_aerotech_test), LiveTable([tomostage.rot]))

Transient Scan ID: 8     Time: 2019-12-13 12:39:20
Persistent Unique Scan ID: 'f78d918c-60fb-466c-b99e-2e18589b3059'



generator repeat ['f78d918c'] (scan num: 8)
Transient Scan ID: 9     Time: 2019-12-13 12:45:05
Persistent Unique Scan ID: '62a9d0dc-bb95-4031-bc72-df33c7912556'


/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/epics/ca.py:1507: UserWarning: ca.get('6idhedm:m7.TDIR') timed out after 1.00 seconds.
  warnings.warn(msg % (name(chid), timeout))
/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/epics/ca.py:1507: UserWarning: ca.get('6idhedm:m7.HLS') timed out after 1.00 seconds.
  warnings.warn(msg % (name(chid), timeout))





generator repeat ['62a9d0dc'] (scan num: 9)
Transient Scan ID: 10     Time: 2019-12-13 12:50:54
Persistent Unique Scan ID: '103c48d6-972e-484c-8908-5a950f5dcecb'



generator repeat ['103c48d6'] (scan num: 10)
Transient Scan ID: 11     Time: 2019-12-13 12:56:39
Persistent Unique Scan ID: '2c9e51fd-f0f8-475f-96ed-a61458e67b07'



generator repeat ['2c9e51fd'] (scan num: 11)
Transient Scan ID: 12     Time: 2019-12-13 13:02:25
Persistent Unique Scan ID: '3c880df0-486d-41d6-b97e-7de8d657087a'



generator repeat ['3c880df0'] (scan num: 12)
Transient Scan ID: 13     Time: 2019-12-13 13:08:12
Persistent Unique Scan ID: '25c066db-44fd-4531-95e6-ec21087a9222'



generator repeat ['25c066db'] (scan num: 13)
Transient Scan ID: 14     Time: 2019-12-13 13:13:57
Persistent Unique Scan ID: 'c7a9ac32-c2a2-4082-b964-f09c7d179506'



generator repeat ['c7a9ac32'] (scan num: 14)
Transient Scan ID: 15     Time: 2019-12-13 13:19:42
Persistent Unique Scan ID: '87b1f3ca-5595-46e4-bff0-08def178cb47'



ge

RunEngineInterrupted: 
Your RunEngine is entering a paused state. These are your options for changing
the state of the RunEngine:

RE.resume()    Resume the plan.
RE.abort()     Perform cleanup, then kill plan. Mark exit_stats='aborted'.
RE.stop()      Perform cleanup, then kill plan. Mark exit_status='success'.
RE.halt()      Emergency Stop: Do not perform cleanup --- just stop.


In [151]:
RE.abort()

Aborting: running cleanup and marking exit_status as 'abort'...



generator repeat ['2939b9e7'] (scan num: 51)
[E 16:50:03.281 run_engine:1469] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1345, in _run
        msg = self._plan_stack[-1].send(resp)
      File "<ipython-input-146-181f55cc9be8>", line 88, in repeat
        yield from test_aerotech_slipring(cfg)
      File "<ipython-input-146-181f55cc9be8>", line 84, in test_aerotech_slipring
        return (yield from scan_closure())
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/utils.py", line 1043, in dec_inner
        return (yield from plan)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/preprocessors.py", line 327, in run_wrapper
        else_plan=close_run)
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/

('f78d918c-60fb-466c-b99e-2e18589b3059',
 '62a9d0dc-bb95-4031-bc72-df33c7912556',
 '103c48d6-972e-484c-8908-5a950f5dcecb',
 '2c9e51fd-f0f8-475f-96ed-a61458e67b07',
 '3c880df0-486d-41d6-b97e-7de8d657087a',
 '25c066db-44fd-4531-95e6-ec21087a9222',
 'c7a9ac32-c2a2-4082-b964-f09c7d179506',
 '87b1f3ca-5595-46e4-bff0-08def178cb47',
 '18ab9cce-52f0-46d1-af62-60b6ff3f1971',
 '1c7e1ebb-bc56-4f36-bce5-5b3dd19f15bf',
 'ccd16be0-a7eb-4f5b-938b-41fca929d5ac',
 '24fb69bc-3a3b-41e6-ae6b-8fafaf7046c8',
 '54c44b5f-56bc-4507-9af3-24adf6b51e9e',
 '59ddcfc2-d388-481d-b89e-78d022150e1d',
 'c60175a1-b290-45e1-ba2a-993edea510a5',
 '6e1d05fc-89d1-4d8b-8f91-23673aca4f34',
 'ae9043fa-96ff-4791-b28e-442e40eb7e7f',
 'ac1e9eef-2ea5-4113-8e97-90e19b36a0bf',
 'f6629f6e-4091-4f59-b9f1-d948d740696e',
 'c9656304-d84f-4ca0-9518-5509cacab137',
 'ae78389b-94b5-4182-b73d-593f1dbea692',
 'eb5d344a-f861-4345-b868-f06c42b77623',
 '1c48f5d6-57c0-41ee-a770-43a4bc59b7f2',
 'f9f545b2-2343-4fed-9691-f08cdd58f1af',
 'e729bf52-2a6b-

In [152]:
tomostage.rot._motor_cal_set.put(1)
tomostage.rot.dial_value.put(tomostage.rot.dial_readback.get()-360)
tomostage.rot._off_value.put(0)
tomostage.rot._motor_cal_set.put(0)

In [154]:
# -- step scan
def reset():
    yield from bps.mv(tomostage.rot._motor_cal_set, 1)
    yield from bps.mv(tomostage.rot.dial_value,     tomostage.rot.dial_readback.get()-360)
    yield from bps.mv(tomostage.rot._off_value,     0)
    yield from bps.mv(tomostage.rot._motor_cal_set, 0)

    
def step_scan(cfg_tomo):
    """
    Collect projections with step motion
    """

    # TODO:
    # the fields need to be updated for 6-ID-D
    yield from bps.mv(det.hdf1.nd_array_port, 'TRANS1')
    yield from bps.mv(det.tiff1.nd_array_port, 'TRANS1') 
    yield from bps.mv(det.trans1.enable, 1)
    yield from bps.mv(det.trans1.enable_filter, 1)
    yield from bps.mv(det.trans1.reset_filter, 1)
    yield from bps.mv(det.trans1.num_filter, cfg_tomo['n_frames'])
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames'])

    angs = np.arange(
        cfg_tomo['omega_start'], 
        cfg_tomo['omega_end']+cfg_tomo['omega_step']/2,
        cfg_tomo['omega_step'],
    )
    for ang in angs:
        yield from bps.checkpoint()
        if ang<-180 or ang>180:
            yield from reset()
        yield from bps.mv(tomostage.rot, ang)
        yield from bps.mv(fast_shutter, 13)
        yield from bps.trigger_and_read([det])
#         yield from bps.mv(fast_shutter, 14)


# -- white field
def collect_white_field(cfg_tomo, atfront=True):
    """
    Collect white/flat field images by moving the sample out of the FOV
    """  
    # move sample out of the way
    # TODO:
    # the details and fields need to be updated for 6-ID-D
    _x = cfg_tomo['fronte_white_kx'] if atfront else cfg_tomo['back_white_kx']
    _z = cfg_tomo['fronte_white_kz'] if atfront else cfg_tomo['back_white_kz']
    yield from bps.mv(tomostage.kx, _x)  #update with correct motor name
    yield from bps.mv(tomostage.kz, _z)

    # setup detector
    # TODO:
    # actual implementation need to be for 6-ID-D
    yield from bps.mv(det.hdf1.nd_array_port, 'TRANS1')
    yield from bps.mv(det.tiff1.nd_array_port, 'TRANS1') 
    yield from bps.mv(det.trans1.enable, 1)
    yield from bps.mv(det.trans1.enable_filter, 1)
    yield from bps.mv(det.trans1.filter_type, 'Average')
    yield from bps.mv(det.trans1.reset_filter, 1)
    yield from bps.mv(det.trans1.num_filter, cfg_tomo['n_frames'])
    yield from bps.mv(det.cam1.trigger_mode, "Software")
    yield from bps.mv(det.cam1.image_mode, "Multiple")
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames']*cfg_tomo['n_white'])
    yield from bps.mv(fast_shutter, 13)
    yield from bps.trigger_and_read([det])
#     yield from bps.mv(fast_shutter, 14)
    # move sample back to FOV
    # NOTE:
    # not sure is this will work or not...
    # TODO:
    #   need to update all the motor names according to StageAero
    yield from bps.mv(tomostage.kx, cfg_tomo['initial_kx'])
    yield from bps.mv(tomostage.kz, cfg_tomo['initial_kz'])
    

# -- dark field
def collect_dark_field(cfg_tomo):
    """
    Collect dark field images by close the shutter
    """
    # TODO:
    #   Need to toggle Fast shutter

    yield from bps.mv(det.hdf1.nd_array_port, 'TRANS1')
    yield from bps.mv(det.tiff1.nd_array_port, 'TRANS1') 
    yield from bps.mv(det.trans1.enable, 1)
    yield from bps.mv(det.trans1.enable_filter, 1)
    yield from bps.mv(det.trans1.reset_filter, 1)
    yield from bps.mv(det.trans1.num_filter, cfg_tomo['n_frames'])
    yield from bps.mv(det.cam1.trigger_mode, "Software")
    yield from bps.mv(det.cam1.image_mode, "Multiple")
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames']*cfg_tomo['n_dark'])
    yield from bps.mv(fast_shutter, 14)
    yield from bps.trigger_and_read([det])


# -- tomo scan
def tomo_scan(cfg):
    """
    Tomography scan plan based on given configuration
    """
    tomostage.kx.move(-1.460)
    tomostage.kz.move(0.030)
    tomostage.ky.move(16.353)
    
    
    fp = cfg['output']['fp']
    fn = cfg['output']['fn']

    #########################
    ## step 0: preparation ##
    #########################
    acquire_time   = cfg['tomo']['acquire_time']
    acquire_period = cfg['tomo']['acquire_period']
    n_white        = cfg['tomo']['n_white']
    n_dark         = cfg['tomo']['n_dark']
    angs = np.arange(
        cfg['tomo']['omega_start'], 
        cfg['tomo']['omega_end']+cfg['tomo']['omega_step']/2,
        cfg['tomo']['omega_step'],
    )
    n_projections = len(angs)
    cfg['tomo']['n_projections'] = n_projections
    cfg['tomo']['total_images']  = n_white + n_projections + n_white + n_dark


    # calculate slew speed for fly scan
    # https://github.com/decarlof/tomo2bm/blob/master/flir/libs/aps2bm_lib.py
    # TODO: considering blue pixels, use 2BM code as ref

    # need to make sure that the sample out position is the same for both front and back
    x0, z0 = tomostage.kx.position, tomostage.kz.position
    dfx, dfz = cfg['tomo']['sample_out_position']['kx'], cfg['tomo']['sample_out_position']['kz']
    rotang = np.radians(cfg['tomo']['omega_end']-cfg['tomo']['omega_start'])
    rotm = np.array([[ np.cos(rotang), np.sin(rotang)],
                        [-np.sin(rotang), np.cos(rotang)]])
    dbxz = np.dot(rotm, np.array([dfx, dfz]))
    dbx = dbxz[0] if abs(dbxz[0]) > 1e-8 else 0.0
    dbz = dbxz[1] if abs(dbxz[1]) > 1e-8 else 0.0
    # now put the value to dict
    cfg['tomo']['initial_kx']       = x0
    cfg['tomo']['initial_kz']       = z0
    cfg['tomo']['fronte_white_kx']  = x0 + dfx
    cfg['tomo']['fronte_white_kz']  = z0 + dfz
    cfg['tomo']['back_white_kx']    = x0 + dbx
    cfg['tomo']['back_white_kz']    = z0 + dbz

    ###############################################
    ## step 0.9: print out the cfg for user info ##
    ###############################################   
    caput("QIMAGE2:TIFF1:AutoSave", 1)
    @bpp.stage_decorator([det])
    @bpp.run_decorator()
    def scan_closure():
        # TODO:
        #   Somewhere we need to check the light status
        # open shutter for beam

        yield from bps.install_suspender(shutter_suspender)
        # config output
        for me in [det.tiff1, det.hdf1]:
            yield from bps.mv(me.file_path, fp)
            yield from bps.mv(me.file_name, fn)
            yield from bps.mv(me.file_write_mode, 2)
            yield from bps.mv(me.num_capture, cfg['tomo']['total_images'])
            yield from bps.mv(me.file_template, ".".join([r"%s%s_%06d",cfg['output']['type'].lower()]))    

        if cfg['output']['type'] in ['tif', 'tiff']:
            yield from bps.mv(det.tiff1.enable, 1)
            yield from bps.mv(det.tiff1.capture, 1)
            yield from bps.mv(det.hdf1.enable, 0)
        elif cfg['output']['type'] in ['hdf', 'hdf1', 'hdf5']:
            yield from bps.mv(det.tiff1.enable, 0)
            yield from bps.mv(det.hdf1.enable, 1)
            yield from bps.mv(det.hdf1.capture, 1)
        else:
            raise ValueError(f"Unsupported output type {cfg['output']['type']}")
        
#         # open main shutter
#         yield from bps.mv(D_shutter, "open")

        # collect front white field
        yield from bps.mv(det.cam1.frame_type, 0)  # for HDF5 dxchange data structure
        yield from collect_white_field(cfg['tomo'], atfront=True)

        # collect projections
        yield from bps.mv(det.cam1.frame_type, 1)  # for HDF5 dxchange data structure
        if cfg['tomo']['type'].lower() == 'step':
            # setting acquire_time and acquire_period
            yield from bps.mv(det.cam1.acquire_time, acquire_time)
            yield from bps.mv(det.cam1.acquire_period, acquire_period)
            # run step_scan
            yield from step_scan(cfg['tomo'])
        else:
            raise ValueError(f"Unsupported scan type: {cfg['tomo']['type']}")

        # collect back white field
        yield from bps.mv(det.cam1.frame_type, 2)  # for HDF5 dxchange data structure
        yield from collect_white_field(cfg['tomo'], atfront=False)

        # collect back dark field
        yield from bps.mv(det.cam1.frame_type, 3)  # for HDF5 dxchange data structure

#         # TODO: no shutter available for Sim testing
#         yield from bps.remove_suspender(shutter_suspender)
#         yield from bps.mv(D_shutter, "close")
#         yield from bps.mv(fast_shutter, 1)

        yield from collect_dark_field(cfg['tomo'])
        
        yield from bps.mv(det_mtr_x, 50)
        yield from bps.mv(tomostage.ky, 6.0)

    return (yield from scan_closure())

In [155]:
def repeat(n_turns, cfg):
    for _ in range(n_turns):
        yield from tomo_scan(cfg)
        yield from reset()

In [157]:
cfg_tomo = {
    'type':    'step',
    'n_white': 5,
    'n_dark':  5,
    'sample_out_position': {
        'kx':  -3,
        'kz':   0,
    },
    'acquire_time':   0.5,
    'acquire_period': 1.0, 
    'omega_step':     10,
    'omega_start':    -180,
    'omega_end':       180,
    'n_frames':       1,   
}

cfg_output = {
    'fn':          'test1',
    'fp':          'Q:\\\\s6idd_dec19\\tomo_HEA1_3\\' ,  # match the ff scan HEA_3
    'type':        'tif',  
}

cfg = {
    'tomo': cfg_tomo,
    'output': cfg_output,
}

In [158]:
RE(repeat(1, cfg), LiveTable([tomostage.rot]))


Transient Scan ID: 52     Time: 2019-12-13 18:45:28
Persistent Unique Scan ID: '4dd9f70e-1a57-4b54-9c35-8c84a9c43e77'



generator repeat ['4dd9f70e'] (scan num: 52)
[E 18:45:33.420 run_engine:1477] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/ophyd/device.py", line 984, in __getattr__
        cpt = self._sig_attrs[name]
    KeyError: 'enable_filter'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "/home/beams/S6HEDM/opt/anaconda3/envs/beamline/lib/python3.7/site-packages/bluesky/run_engine.py", line 1345, in _run
        msg = self._plan_stack[-1].send(resp)
      File "<ipython-input-155-d718638217e7>", line 3, in repeat
        yield from tomo_scan(cfg)
      File "<ipython-input-154-cb2a0e923d00>", line 215, in tomo_scan
        return (yield from scan_closure())
      File "/home/beams/S6HEDM/opt/anaconda3/

AttributeError: enable_filter

In [159]:
# -- step scan
def reset():
    yield from bps.mv(tomostage.rot._motor_cal_set, 1)
    yield from bps.mv(tomostage.rot.dial_value,     tomostage.rot.dial_readback.get()-360)
    yield from bps.mv(tomostage.rot._off_value,     0)
    yield from bps.mv(tomostage.rot._motor_cal_set, 0)

    
def step_scan(cfg_tomo):
    """
    Collect projections with step motion
    """

    # TODO:
    # the fields need to be updated for 6-ID-D
    yield from bps.mv(det.hdf1.nd_array_port, 'TRANS1')
    yield from bps.mv(det.tiff1.nd_array_port, 'TRANS1') 
    yield from bps.mv(det.trans1.enable, 1)
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames'])

    angs = np.arange(
        cfg_tomo['omega_start'], 
        cfg_tomo['omega_end']+cfg_tomo['omega_step']/2,
        cfg_tomo['omega_step'],
    )
    for ang in angs:
        yield from bps.checkpoint()
        if ang<-180 or ang>180:
            yield from reset()
        yield from bps.mv(tomostage.rot, ang)
        yield from bps.mv(fast_shutter, 13)
        yield from bps.trigger_and_read([det])
#         yield from bps.mv(fast_shutter, 14)


# -- white field
def collect_white_field(cfg_tomo, atfront=True):
    """
    Collect white/flat field images by moving the sample out of the FOV
    """  
    # move sample out of the way
    # TODO:
    # the details and fields need to be updated for 6-ID-D
    _x = cfg_tomo['fronte_white_kx'] if atfront else cfg_tomo['back_white_kx']
    _z = cfg_tomo['fronte_white_kz'] if atfront else cfg_tomo['back_white_kz']
    yield from bps.mv(tomostage.kx, _x)  #update with correct motor name
    yield from bps.mv(tomostage.kz, _z)

    # setup detector
    # TODO:
    # actual implementation need to be for 6-ID-D
    yield from bps.mv(det.hdf1.nd_array_port, 'TRANS1')
    yield from bps.mv(det.tiff1.nd_array_port, 'TRANS1') 
    yield from bps.mv(det.trans1.enable, 1)
    yield from bps.mv(det.cam1.trigger_mode, "Software")
    yield from bps.mv(det.cam1.image_mode, "Multiple")
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames']*cfg_tomo['n_white'])
    yield from bps.mv(fast_shutter, 13)
    yield from bps.trigger_and_read([det])
#     yield from bps.mv(fast_shutter, 14)
    # move sample back to FOV
    # NOTE:
    # not sure is this will work or not...
    # TODO:
    #   need to update all the motor names according to StageAero
    yield from bps.mv(tomostage.kx, cfg_tomo['initial_kx'])
    yield from bps.mv(tomostage.kz, cfg_tomo['initial_kz'])
    

# -- dark field
def collect_dark_field(cfg_tomo):
    """
    Collect dark field images by close the shutter
    """
    # TODO:
    #   Need to toggle Fast shutter

    yield from bps.mv(det.hdf1.nd_array_port, 'TRANS1')
    yield from bps.mv(det.tiff1.nd_array_port, 'TRANS1') 
    yield from bps.mv(det.trans1.enable, 1)
    yield from bps.mv(det.cam1.trigger_mode, "Software")
    yield from bps.mv(det.cam1.image_mode, "Multiple")
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames']*cfg_tomo['n_dark'])
    yield from bps.mv(fast_shutter, 14)
    yield from bps.trigger_and_read([det])


# -- tomo scan
def tomo_scan(cfg):
    """
    Tomography scan plan based on given configuration
    """
    tomostage.kx.move(-1.460)
    tomostage.kz.move(0.030)
    tomostage.ky.move(16.353)
    
    
    fp = cfg['output']['fp']
    fn = cfg['output']['fn']

    #########################
    ## step 0: preparation ##
    #########################
    acquire_time   = cfg['tomo']['acquire_time']
    acquire_period = cfg['tomo']['acquire_period']
    n_white        = cfg['tomo']['n_white']
    n_dark         = cfg['tomo']['n_dark']
    angs = np.arange(
        cfg['tomo']['omega_start'], 
        cfg['tomo']['omega_end']+cfg['tomo']['omega_step']/2,
        cfg['tomo']['omega_step'],
    )
    n_projections = len(angs)
    cfg['tomo']['n_projections'] = n_projections
    cfg['tomo']['total_images']  = n_white + n_projections + n_white + n_dark


    # calculate slew speed for fly scan
    # https://github.com/decarlof/tomo2bm/blob/master/flir/libs/aps2bm_lib.py
    # TODO: considering blue pixels, use 2BM code as ref

    # need to make sure that the sample out position is the same for both front and back
    x0, z0 = tomostage.kx.position, tomostage.kz.position
    dfx, dfz = cfg['tomo']['sample_out_position']['kx'], cfg['tomo']['sample_out_position']['kz']
    rotang = np.radians(cfg['tomo']['omega_end']-cfg['tomo']['omega_start'])
    rotm = np.array([[ np.cos(rotang), np.sin(rotang)],
                        [-np.sin(rotang), np.cos(rotang)]])
    dbxz = np.dot(rotm, np.array([dfx, dfz]))
    dbx = dbxz[0] if abs(dbxz[0]) > 1e-8 else 0.0
    dbz = dbxz[1] if abs(dbxz[1]) > 1e-8 else 0.0
    # now put the value to dict
    cfg['tomo']['initial_kx']       = x0
    cfg['tomo']['initial_kz']       = z0
    cfg['tomo']['fronte_white_kx']  = x0 + dfx
    cfg['tomo']['fronte_white_kz']  = z0 + dfz
    cfg['tomo']['back_white_kx']    = x0 + dbx
    cfg['tomo']['back_white_kz']    = z0 + dbz

    ###############################################
    ## step 0.9: print out the cfg for user info ##
    ###############################################   
    caput("QIMAGE2:TIFF1:AutoSave", 1)
    @bpp.stage_decorator([det])
    @bpp.run_decorator()
    def scan_closure():
        # TODO:
        #   Somewhere we need to check the light status
        # open shutter for beam

#         yield from bps.install_suspender(shutter_suspender)
        # config output
        for me in [det.tiff1, det.hdf1]:
            yield from bps.mv(me.file_path, fp)
            yield from bps.mv(me.file_name, fn)
            yield from bps.mv(me.file_write_mode, 2)
            yield from bps.mv(me.num_capture, cfg['tomo']['total_images'])
            yield from bps.mv(me.file_template, ".".join([r"%s%s_%06d",cfg['output']['type'].lower()]))    

        if cfg['output']['type'] in ['tif', 'tiff']:
            yield from bps.mv(det.tiff1.enable, 1)
            yield from bps.mv(det.tiff1.capture, 1)
            yield from bps.mv(det.hdf1.enable, 0)
        elif cfg['output']['type'] in ['hdf', 'hdf1', 'hdf5']:
            yield from bps.mv(det.tiff1.enable, 0)
            yield from bps.mv(det.hdf1.enable, 1)
            yield from bps.mv(det.hdf1.capture, 1)
        else:
            raise ValueError(f"Unsupported output type {cfg['output']['type']}")
        
#         # open main shutter
#         yield from bps.mv(D_shutter, "open")

        # collect front white field
        yield from bps.mv(det.cam1.frame_type, 0)  # for HDF5 dxchange data structure
        yield from collect_white_field(cfg['tomo'], atfront=True)

        # collect projections
        yield from bps.mv(det.cam1.frame_type, 1)  # for HDF5 dxchange data structure
        if cfg['tomo']['type'].lower() == 'step':
            # setting acquire_time and acquire_period
            yield from bps.mv(det.cam1.acquire_time, acquire_time)
            yield from bps.mv(det.cam1.acquire_period, acquire_period)
            # run step_scan
            yield from step_scan(cfg['tomo'])
        else:
            raise ValueError(f"Unsupported scan type: {cfg['tomo']['type']}")

        # collect back white field
        yield from bps.mv(det.cam1.frame_type, 2)  # for HDF5 dxchange data structure
        yield from collect_white_field(cfg['tomo'], atfront=False)

        # collect back dark field
        yield from bps.mv(det.cam1.frame_type, 3)  # for HDF5 dxchange data structure

#         # TODO: no shutter available for Sim testing
#         yield from bps.remove_suspender(shutter_suspender)
#         yield from bps.mv(D_shutter, "close")
#         yield from bps.mv(fast_shutter, 1)

        yield from collect_dark_field(cfg['tomo'])
        
        yield from bps.mv(det_mtr_x, 50)
        yield from bps.mv(tomostage.ky, 6.0)

    return (yield from scan_closure())

Suspender SuspendFloor(EpicsSignalRO(read_pv='PA:06ID:STA_D_SDS_OPEN_PL', name='D_shutter_pss_state', parent='D_shutter', value=1, timestamp=1576284484.167936, pv_kw={}, auto_monitor=False, string=False), sleep=0, pre_plan=None, post_plan=None,tripped_message=) reports a return to nominal conditions. Will sleep for 0 seconds and then release suspension at 2019-12-13 18:48:04.


In [160]:
RE.remove_suspender(shutter_suspender)

In [161]:
RE(repeat(1, cfg), LiveTable([tomostage.rot]))

Transient Scan ID: 53     Time: 2019-12-13 18:49:20
Persistent Unique Scan ID: '8d3254dc-52a5-465e-86a8-adea96a612c6'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 18:49:30.8 |
|         1 | 18:49:30.8 |
|         2 | 18:49:36.6 |
|         2 | 18:49:36.6 |
|         3 | 18:49:40.1 |
|         3 | 18:49:40.1 |
|         4 | 18:49:43.6 |
|         4 | 18:49:43.6 |
|         5 | 18:49:47.1 |
|         5 | 18:49:47.1 |
|         6 | 18:49:50.6 |
|         6 | 18:49:50.6 |
|         7 | 18:49:54.1 |
|         7 | 18:49:54.1 |
|         8 | 18:49:57.6 |
|         8 | 18:49:57.6 |
|         9 | 18:50:01.1 |
|         9 | 18:50:01.1 |
|        10 | 18:50:04.6 |
|        10 | 18:50:04.6 |
|        11 | 18:50:08.1 |
|        11 | 18:50:08.1 |
|        12 | 18:50:11.6 |
|        12 | 18:50:11.6 |
|        13 | 18:50:15.1 |
|        13 | 18:50:15.

NameError: name 'det_mtr_x' is not defined

In [162]:
# -- step scan
def reset():
    yield from bps.mv(tomostage.rot._motor_cal_set, 1)
    yield from bps.mv(tomostage.rot.dial_value,     tomostage.rot.dial_readback.get()-360)
    yield from bps.mv(tomostage.rot._off_value,     0)
    yield from bps.mv(tomostage.rot._motor_cal_set, 0)

    
def step_scan(cfg_tomo):
    """
    Collect projections with step motion
    """

    # TODO:
    # the fields need to be updated for 6-ID-D
    yield from bps.mv(det.hdf1.nd_array_port, 'TRANS1')
    yield from bps.mv(det.tiff1.nd_array_port, 'TRANS1') 
    yield from bps.mv(det.trans1.enable, 1)
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames'])

    angs = np.arange(
        cfg_tomo['omega_start'], 
        cfg_tomo['omega_end']+cfg_tomo['omega_step']/2,
        cfg_tomo['omega_step'],
    )
    for ang in angs:
        yield from bps.checkpoint()
        if ang<-180 or ang>180:
            yield from reset()
        yield from bps.mv(tomostage.rot, ang)
        yield from bps.mv(fast_shutter, 13)
        yield from bps.trigger_and_read([det])
#         yield from bps.mv(fast_shutter, 14)


# -- white field
def collect_white_field(cfg_tomo, atfront=True):
    """
    Collect white/flat field images by moving the sample out of the FOV
    """  
    # move sample out of the way
    # TODO:
    # the details and fields need to be updated for 6-ID-D
    _x = cfg_tomo['fronte_white_kx'] if atfront else cfg_tomo['back_white_kx']
    _z = cfg_tomo['fronte_white_kz'] if atfront else cfg_tomo['back_white_kz']
    yield from bps.mv(tomostage.kx, _x)  #update with correct motor name
    yield from bps.mv(tomostage.kz, _z)

    # setup detector
    # TODO:
    # actual implementation need to be for 6-ID-D
    yield from bps.mv(det.hdf1.nd_array_port, 'TRANS1')
    yield from bps.mv(det.tiff1.nd_array_port, 'TRANS1') 
    yield from bps.mv(det.trans1.enable, 1)
    yield from bps.mv(det.cam1.trigger_mode, "Software")
    yield from bps.mv(det.cam1.image_mode, "Multiple")
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames']*cfg_tomo['n_white'])
    yield from bps.mv(fast_shutter, 13)
    yield from bps.trigger_and_read([det])
#     yield from bps.mv(fast_shutter, 14)
    # move sample back to FOV
    # NOTE:
    # not sure is this will work or not...
    # TODO:
    #   need to update all the motor names according to StageAero
    yield from bps.mv(tomostage.kx, cfg_tomo['initial_kx'])
    yield from bps.mv(tomostage.kz, cfg_tomo['initial_kz'])
    

# -- dark field
def collect_dark_field(cfg_tomo):
    """
    Collect dark field images by close the shutter
    """
    # TODO:
    #   Need to toggle Fast shutter

    yield from bps.mv(det.hdf1.nd_array_port, 'TRANS1')
    yield from bps.mv(det.tiff1.nd_array_port, 'TRANS1') 
    yield from bps.mv(det.trans1.enable, 1)
    yield from bps.mv(det.cam1.trigger_mode, "Software")
    yield from bps.mv(det.cam1.image_mode, "Multiple")
    yield from bps.mv(det.cam1.num_images, cfg_tomo['n_frames']*cfg_tomo['n_dark'])
    yield from bps.mv(fast_shutter, 14)
    yield from bps.trigger_and_read([det])


# -- tomo scan
def tomo_scan(cfg):
    """
    Tomography scan plan based on given configuration
    """
    tomostage.kx.move(-1.460)
    tomostage.kz.move(0.030)
    tomostage.ky.move(16.353)
    
    
    fp = cfg['output']['fp']
    fn = cfg['output']['fn']

    #########################
    ## step 0: preparation ##
    #########################
    acquire_time   = cfg['tomo']['acquire_time']
    acquire_period = cfg['tomo']['acquire_period']
    n_white        = cfg['tomo']['n_white']
    n_dark         = cfg['tomo']['n_dark']
    angs = np.arange(
        cfg['tomo']['omega_start'], 
        cfg['tomo']['omega_end']+cfg['tomo']['omega_step']/2,
        cfg['tomo']['omega_step'],
    )
    n_projections = len(angs)
    cfg['tomo']['n_projections'] = n_projections
    cfg['tomo']['total_images']  = n_white + n_projections + n_white + n_dark


    # calculate slew speed for fly scan
    # https://github.com/decarlof/tomo2bm/blob/master/flir/libs/aps2bm_lib.py
    # TODO: considering blue pixels, use 2BM code as ref

    # need to make sure that the sample out position is the same for both front and back
    x0, z0 = tomostage.kx.position, tomostage.kz.position
    dfx, dfz = cfg['tomo']['sample_out_position']['kx'], cfg['tomo']['sample_out_position']['kz']
    rotang = np.radians(cfg['tomo']['omega_end']-cfg['tomo']['omega_start'])
    rotm = np.array([[ np.cos(rotang), np.sin(rotang)],
                        [-np.sin(rotang), np.cos(rotang)]])
    dbxz = np.dot(rotm, np.array([dfx, dfz]))
    dbx = dbxz[0] if abs(dbxz[0]) > 1e-8 else 0.0
    dbz = dbxz[1] if abs(dbxz[1]) > 1e-8 else 0.0
    # now put the value to dict
    cfg['tomo']['initial_kx']       = x0
    cfg['tomo']['initial_kz']       = z0
    cfg['tomo']['fronte_white_kx']  = x0 + dfx
    cfg['tomo']['fronte_white_kz']  = z0 + dfz
    cfg['tomo']['back_white_kx']    = x0 + dbx
    cfg['tomo']['back_white_kz']    = z0 + dbz

    ###############################################
    ## step 0.9: print out the cfg for user info ##
    ###############################################   
    caput("QIMAGE2:TIFF1:AutoSave", 1)
    @bpp.stage_decorator([det])
    @bpp.run_decorator()
    def scan_closure():
        # TODO:
        #   Somewhere we need to check the light status
        # open shutter for beam

#         yield from bps.install_suspender(shutter_suspender)
        # config output
        for me in [det.tiff1, det.hdf1]:
            yield from bps.mv(me.file_path, fp)
            yield from bps.mv(me.file_name, fn)
            yield from bps.mv(me.file_write_mode, 2)
            yield from bps.mv(me.num_capture, cfg['tomo']['total_images'])
            yield from bps.mv(me.file_template, ".".join([r"%s%s_%06d",cfg['output']['type'].lower()]))    

        if cfg['output']['type'] in ['tif', 'tiff']:
            yield from bps.mv(det.tiff1.enable, 1)
            yield from bps.mv(det.tiff1.capture, 1)
            yield from bps.mv(det.hdf1.enable, 0)
        elif cfg['output']['type'] in ['hdf', 'hdf1', 'hdf5']:
            yield from bps.mv(det.tiff1.enable, 0)
            yield from bps.mv(det.hdf1.enable, 1)
            yield from bps.mv(det.hdf1.capture, 1)
        else:
            raise ValueError(f"Unsupported output type {cfg['output']['type']}")
        
#         # open main shutter
#         yield from bps.mv(D_shutter, "open")

        # collect front white field
        yield from bps.mv(det.cam1.frame_type, 0)  # for HDF5 dxchange data structure
        yield from collect_white_field(cfg['tomo'], atfront=True)

        # collect projections
        yield from bps.mv(det.cam1.frame_type, 1)  # for HDF5 dxchange data structure
        if cfg['tomo']['type'].lower() == 'step':
            # setting acquire_time and acquire_period
            yield from bps.mv(det.cam1.acquire_time, acquire_time)
            yield from bps.mv(det.cam1.acquire_period, acquire_period)
            # run step_scan
            yield from step_scan(cfg['tomo'])
        else:
            raise ValueError(f"Unsupported scan type: {cfg['tomo']['type']}")

        # collect back white field
        yield from bps.mv(det.cam1.frame_type, 2)  # for HDF5 dxchange data structure
        yield from collect_white_field(cfg['tomo'], atfront=False)

        # collect back dark field
        yield from bps.mv(det.cam1.frame_type, 3)  # for HDF5 dxchange data structure

#         # TODO: no shutter available for Sim testing
#         yield from bps.remove_suspender(shutter_suspender)
#         yield from bps.mv(D_shutter, "close")
#         yield from bps.mv(fast_shutter, 1)

        yield from collect_dark_field(cfg['tomo'])
        
#         yield from bps.mv(det_mtr_x, 50)
#         yield from bps.mv(tomostage.ky, 6.0)

    return (yield from scan_closure())

In [163]:
def force_reset():
    tomostage.rot._motor_cal_set.put(1)
    #_current_rot_pos = tomostage.rot._dialreadback.get()
    tomostage.rot.dial_value.put(tomostage.rot.dial_readback.get()-360)
    tomostage.rot._off_value.put(0)
    tomostage.rot._motor_cal_set.put(0)

In [164]:
force_reset()

In [165]:
RE(repeat(1, cfg), LiveTable([tomostage.rot]))

Transient Scan ID: 54     Time: 2019-12-13 18:55:49
Persistent Unique Scan ID: '439f4f05-bc92-4f7e-9e9b-878e80425f9e'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 18:56:00.0 |
|         1 | 18:56:00.0 |
|         2 | 18:56:05.8 |
|         2 | 18:56:05.8 |
|         3 | 18:56:09.3 |
|         3 | 18:56:09.3 |
|         4 | 18:56:12.8 |
|         4 | 18:56:12.8 |
|         5 | 18:56:16.3 |
|         5 | 18:56:16.3 |
|         6 | 18:56:19.8 |
|         6 | 18:56:19.8 |
|         7 | 18:56:23.3 |
|         7 | 18:56:23.3 |
|         8 | 18:56:26.8 |
|         8 | 18:56:26.8 |
|         9 | 18:56:30.3 |
|         9 | 18:56:30.3 |
|        10 | 18:56:33.8 |
|        10 | 18:56:33.8 |
|        11 | 18:56:37.3 |
|        11 | 18:56:37.3 |
|        12 | 18:56:40.8 |
|        12 | 18:56:40.8 |
|        13 | 18:56:44.4 |
|        13 | 18:56:44.

('439f4f05-bc92-4f7e-9e9b-878e80425f9e',)

CeO2

In [166]:
det.cam1.acquire.put(1)